# IA - Projeto 01: Horário de turma 

# 1. Introdução
### Membros do grupo
- Hugo Ferreira Baptista — nº 23279
- Nuno da Cunha Faria Gajo — nº 23002


### Contexto e Objetivo do projeto
O objetivo deste projeto é para desenvolver um **agente inteligente** capaz de gerar **horários de turmas** que respeitem certas restrições tais como, disponibilidade de professores, salas e evitar conflitos de horários.  
O problema é formulado como um **CSP** utilizando Python e a biblioteca `python-constraint`.


In [9]:
# Install contraint library
%pip install python-constraint

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Import contraint library
from constraint import *
from itertools import combinations
from collections import defaultdict
import random

# 2. Design do Agente

Nesta seção, definimos o problema do horário de turmas como um **CSP**.

Cada variável representa uma **aula** e pode assumir um valor (bloco de tempo, sala, online) onde:
- `bloco de tempo` \ `block` ∈ [1; 20] (20 blocos por semana);
- `sala` \ `room` ∈ {R1, R2, R3, Lab01};
- `online` ∈ {Verdadeiro, Falso}.

Nós definimos:
- Variáveis e domínios;  
- Restrições rígidas (obrigatórias);
- Restrições flexíveis (opcionais, tratadas posteriormente);
- Heurísticas para melhorar o desempenho;

In [11]:
# -----------------------------
# DADOS INICIAIS
# -----------------------------

# Classes e suas UCs
classes = {
    "t01": ["UC11","UC12","UC13","UC14","UC15"],
    "t02": ["UC21","UC22","UC23","UC24","UC25"],
    "t03": ["UC31","UC32","UC33","UC34","UC35"]
}

# Professores e UCs que lecionam
teachers = {
    "jo": ["UC11","UC21","UC22","UC31"],
    "mike": ["UC12","UC23","UC32"],
    "rob": ["UC13","UC14","UC24","UC33"],
    "sue": ["UC15","UC25","UC34","UC35"]
}

# Restrição de horários em que o professor NÃO pode dar aula
teacher_restrictions = {
    "mike": [13,14,15,16,17,18,19,20],
    "rob": [1,2,3,4],
    "sue": [9,10,11,12,17,18,19,20]
}

# Salas fixas para algumas UCs
fixed_rooms = {"UC14":"Lab01","UC22":"Lab01"}

# Aulas online permitidas
online_allowed = {"UC21","UC31"}

# Lista de todas as salas disponíveis
rooms = ["R1","R2","R3","Lab01"]

# Configuração dos slots
slots_per_day = 4
total_days = 5
total_slots = slots_per_day * total_days  # 20 slots na semana

# -----------------------------
# FUNÇÕES AUXILIARES
# -----------------------------

# Determina o dia a partir do slot
def dia(slot): 
    return (slot-1)//slots_per_day + 1

# Determina a hora dentro do dia a partir do slot
def hora(slot): 
    return (slot-1)%slots_per_day + 1

# Funções de hard constraint
def no_overlap(a,b): 
    """Garante que duas aulas não ocorram ao mesmo tempo (mesmo slot)"""
    return a[0] != b[0]

def no_room_conflict(a,b): 
    """Garante que duas aulas não ocorram na mesma sala ao mesmo tempo"""
    return not (a[0]==b[0] and a[1]==b[1])

# Função de soft constraint: calcula pontuação de uma turma
def soft_score_per_turma(solution, turma_ucs):
    score = 0
    dias_turma = set()
    aulas_dia = defaultdict(list)
    
    for uc in turma_ucs:
        dias = [dia(solution[f"{uc}_{i}"][0]) for i in [1,2]]  # dias das aulas da UC
        dias_turma.update(dias)
        
        # Soft constraint: aulas da mesma UC em dias distintos
        if len(set(dias)) == 2:
            score += 5
        else:
            score -= 3
        
        # Organizar aulas por dia para verificar consecutividade
        for i in [1,2]:
            s = solution[f"{uc}_{i}"][0]
            aulas_dia[dia(s)].append(hora(s))
    
    # Soft constraint: turma distribuída em no máximo 4 dias
    if len(dias_turma) <= 4:
        score += 7
    else:
        score -= (len(dias_turma)-4)*2
    
    # Soft constraint: aulas consecutivas dentro do dia
    for hs in aulas_dia.values():
        hs.sort()
        gaps = sum((b - a) > 1 for a,b in zip(hs, hs[1:]))
        score -= gaps * 2
    
    return score

# -----------------------------
# CRIAR PROBLEMA
# -----------------------------
problem = Problem()

# -----------------------------
# ADICIONAR VARIÁVEIS E DOMÍNIOS
# -----------------------------
for turma, ucs in classes.items():
    for uc in ucs:
        teacher = next(t for t,c in teachers.items() if uc in c)
        unavailable = teacher_restrictions.get(teacher, [])
        
        for i in [1,2]:  # duas aulas por UC
            var = f"{uc}_{i}"  # nome da variável
            domain = []
            
            # Construir domínio possível para cada aula
            for slot in range(1, total_slots+1):
                if slot in unavailable: 
                    continue  # professor indisponível
                
                for room in rooms:
                    if uc in fixed_rooms and room != fixed_rooms[uc]: 
                        continue  # sala fixa
                    for online in ([True, False] if uc in online_allowed else [False]):
                        domain.append((slot, room, online))
            
            random.shuffle(domain)  # embaralhar para variar soluções
            problem.addVariable(var, domain)

# -----------------------------
# ADICIONAR CONSTRAINTS INTERNAS POR TURMA
# -----------------------------
for turma, ucs in classes.items():
    vars_turma = [f"{uc}_{i}" for uc in ucs for i in [1,2]]
    for i in range(len(vars_turma)):
        for j in range(i+1, len(vars_turma)):
            problem.addConstraint(no_overlap, (vars_turma[i], vars_turma[j]))
            problem.addConstraint(no_room_conflict, (vars_turma[i], vars_turma[j]))

# -----------------------------
# ADICIONAR CONSTRAINTS GLOBAIS (entre turmas)
# -----------------------------
all_vars = [f"{uc}_{i}" for turma in classes.values() for uc in turma for i in [1,2]]
for i in range(len(all_vars)):
    for j in range(i+1, len(all_vars)):
        var1, var2 = all_vars[i], all_vars[j]
        uc1, uc2 = var1[:4], var2[:4]
        prof1 = next(t for t,c in teachers.items() if uc1 in c)
        prof2 = next(t for t,c in teachers.items() if uc2 in c)
        
        if prof1 == prof2:
            problem.addConstraint(no_overlap, (var1, var2))  # professor não pode dar duas aulas ao mesmo tempo
        problem.addConstraint(no_room_conflict, (var1, var2))  # mesma sala não ao mesmo tempo


# 3. Agente em execução
Nesta seção, geramos várias soluções para o problema (limite de 1000 soluções) e escolhemos a que possuir **melhor pontuação**.

O **horário** é mostrado em tabelas, uma para cada turma.

In [12]:
# -----------------------------
# GERAR SOLUÇÕES LIMITADAS
# -----------------------------
solutions = []
sol = problem.getSolution()  # pega a primeira solução
tries = 0
while sol and tries < 1000:  # limitar tentativas para não travar
    solutions.append(sol)
    sol = problem.getSolution()
    tries += 1

print(f"{len(solutions)} soluções geradas")

# -----------------------------
# ESCOLHER MELHOR SOLUÇÃO SEGUNDO SOFT CONSTRAINTS
# -----------------------------
if not solutions:
    print("Nenhuma solução encontrada")
else:
    # calcula pontuação total somando todas as turmas
    best_solution = max(solutions, key=lambda s: sum(soft_score_per_turma(s, ucs) for ucs in classes.values()))
    print("\nSolução encontrada\n")
    
    total_score = 0
    horas = ["09-11","11-13","14-16","16-18"]
    
    # -----------------------------
    # IMPRIMIR HORÁRIOS COM PONTUAÇÃO
    # -----------------------------
    for turma, ucs in classes.items():
        turma_score = soft_score_per_turma(best_solution, ucs)
        total_score += turma_score
        
        print(f"\n==============================")
        print(f" Horário da Turma {turma.upper()} (Pontuação: {turma_score})")
        print(f"==============================")
        
        # Criar tabela vazia
        tabela = {d:{h:"" for h in horas} for d in range(1,6)}
        
        for uc in ucs:
            for i in [1,2]:
                var = f"{uc}_{i}"
                slot, room, online = best_solution[var]
                d = dia(slot)
                h = horas[(slot-1)%4]
                modo = "ON" if online else "PR"
                tabela[d][h] = f"{uc} ({room}, {modo})"
        
        # Imprimir tabela formatada
        print(f"{'Hora':<10}  Dia1              Dia2              Dia3              Dia4              Dia5")
        print("-"*95)
        for h in horas:
            linha = f"{h:<10}  "
            for d in range(1,6):
                cel = tabela[d][h] if tabela[d][h] else "-"
                linha += f"{cel:<18} "
            print(linha)
    
    print(f"\n Pontuação total da solução: {total_score} pontos\n")


1000 soluções geradas

Solução encontrada


 Horário da Turma T01 (Pontuação: 30)
Hora        Dia1              Dia2              Dia3              Dia4              Dia5
-----------------------------------------------------------------------------------------------
09-11       -                  UC13 (R1, PR)      -                  UC15 (R1, PR)      -                  
11-13       -                  UC15 (R1, PR)      -                  -                  UC11 (Lab01, PR)   
14-16       -                  UC12 (Lab01, PR)   UC12 (R3, PR)      UC11 (R3, PR)      UC13 (R2, PR)      
16-18       -                  UC14 (Lab01, PR)   UC14 (Lab01, PR)   -                  -                  

 Horário da Turma T02 (Pontuação: 1)
Hora        Dia1              Dia2              Dia3              Dia4              Dia5
-----------------------------------------------------------------------------------------------
09-11       UC21 (R1, ON)      UC22 (Lab01, PR)   UC23 (Lab01, PR)   -        

# 4. Conclusão
Neste trabalho, desenvolvemos um agente inteligente utilizando **Python** e a biblioteca `python-constraint`, capaz de gerar horários para múltiplas turmas respeitando hard constraints e considerando soft constraints para melhorar a qualidade do horário.

As hard constraints garantiram que professores e salas não entram em conflito, respeitando restrições de horários e salas fixas. 

As soft constraints permitiram aulas em dias distintos, dias compactos por turma e aulas consecutivas, sendo avaliadas por uma pontuação que orienta a escolha da melhor solução.

Como resultado, foi possível gerar soluções consistentes para todas as turmas, demonstrando que CSPs em Python são eficazes para problemas de criação de horários, equilibrando viabilidade e qualidade do horário.